# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 8:33AM,245398,10,NST-1110,"Amcyte Pharma, Inc.",Released
1,Aug 31 2022 8:33AM,245397,22,083122,"NBTY Global, Inc.",Released
2,Aug 31 2022 8:32AM,245394,10,8462485,MedStone Pharma LLC,Released
3,Aug 31 2022 8:32AM,245394,10,8462486,MedStone Pharma LLC,Released
4,Aug 31 2022 8:32AM,245394,10,8462487,MedStone Pharma LLC,Released
5,Aug 31 2022 8:32AM,245394,10,8462488,MedStone Pharma LLC,Released
6,Aug 31 2022 8:32AM,245394,10,8462489,MedStone Pharma LLC,Released
7,Aug 31 2022 8:32AM,245394,10,8462490,MedStone Pharma LLC,Released
8,Aug 31 2022 8:32AM,245394,10,8462491,MedStone Pharma LLC,Released
9,Aug 31 2022 8:32AM,245394,10,8462492,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,245392,Released,1
29,245393,Released,8
30,245394,Released,11
31,245397,Released,1
32,245398,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245392,NaN,NaN,1.0
245393,NaN,NaN,8.0
245394,NaN,NaN,11.0
245397,NaN,NaN,1.0
245398,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0.0,0.0,66.0
245276,0.0,0.0,1.0
245277,0.0,0.0,1.0
245280,0.0,1.0,0.0
245281,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0,0,66
245276,0,0,1
245277,0,0,1
245280,0,1,0
245281,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,0,0,66
1,245276,0,0,1
2,245277,0,0,1
3,245280,0,1,0
4,245281,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,,,66
1,245276,,,1
2,245277,,,1
3,245280,,1,
4,245281,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc."
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc."
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC
13,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC"
14,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.
23,Aug 31 2022 8:28AM,245393,10,"Snap Medical Industries, LLC"
31,Aug 31 2022 8:27AM,245392,19,Else Nutrition
32,Aug 31 2022 8:25AM,245390,10,ISDIN Corporation
33,Aug 31 2022 8:20AM,245389,10,Emerginnova
38,Aug 31 2022 8:19AM,245387,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc.",,,1
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc.",,,1
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC,,,11
3,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC",,,1
4,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.,,,9
5,Aug 31 2022 8:28AM,245393,10,"Snap Medical Industries, LLC",,,8
6,Aug 31 2022 8:27AM,245392,19,Else Nutrition,,,1
7,Aug 31 2022 8:25AM,245390,10,ISDIN Corporation,,,1
8,Aug 31 2022 8:20AM,245389,10,Emerginnova,,,5
9,Aug 31 2022 8:19AM,245387,10,ISDIN Corporation,,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc.",1,,
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc.",1,,
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC,11,,
3,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC",1,,
4,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.,9,,
5,Aug 31 2022 8:28AM,245393,10,"Snap Medical Industries, LLC",8,,
6,Aug 31 2022 8:27AM,245392,19,Else Nutrition,1,,
7,Aug 31 2022 8:25AM,245390,10,ISDIN Corporation,1,,
8,Aug 31 2022 8:20AM,245389,10,Emerginnova,5,,
9,Aug 31 2022 8:19AM,245387,10,ISDIN Corporation,20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc.",1,,
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc.",1,,
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC,11,,
3,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC",1,,
4,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC,11.0,NaN,NaN
3,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC",1.0,NaN,NaN
4,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 8:33AM,245398,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
1,Aug 31 2022 8:33AM,245397,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Aug 31 2022 8:32AM,245394,10,MedStone Pharma LLC,11.0,0.0,0.0
3,Aug 31 2022 8:30AM,245380,10,"Akron BioProducts, LLC",1.0,0.0,0.0
4,Aug 31 2022 8:30AM,245382,10,Eywa Pharma Inc.,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2208425,57.0,0.0,0.0
15,490282,78.0,0.0,34.0
19,736138,12.0,15.0,8.0
20,981356,62.0,8.0,16.0
21,2207711,6.0,3.0,0.0
22,245397,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2208425,57.0,0.0,0.0
1,15,490282,78.0,0.0,34.0
2,19,736138,12.0,15.0,8.0
3,20,981356,62.0,8.0,16.0
4,21,2207711,6.0,3.0,0.0
5,22,245397,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,0.0,0.0
1,15,78.0,0.0,34.0
2,19,12.0,15.0,8.0
3,20,62.0,8.0,16.0
4,21,6.0,3.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,15,Released,78.0
2,19,Released,12.0
3,20,Released,62.0
4,21,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,34.0,8.0,16.0,0.0,0.0
Executing,0.0,0.0,15.0,8.0,3.0,0.0
Released,57.0,78.0,12.0,62.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,34.0,8.0,16.0,0.0,0.0
1,Executing,0.0,0.0,15.0,8.0,3.0,0.0
2,Released,57.0,78.0,12.0,62.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,34.0,8.0,16.0,0.0,0.0
1,Executing,0.0,0.0,15.0,8.0,3.0,0.0
2,Released,57.0,78.0,12.0,62.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()